<a href="https://colab.research.google.com/github/MuhammedAshraf2020/IMDB-Sentimal-Analysis-LSTM/blob/main/Sentimal_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Data

In [1]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
import string   
import re

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 47% 12.0M/25.7M [00:00<00:00, 122MB/s]
100% 25.7M/25.7M [00:00<00:00, 164MB/s]


In [6]:
!unzip /content/imdb-dataset-of-50k-movie-reviews.zip

Archive:  /content/imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [7]:
dataset = pd.read_csv("/content/IMDB Dataset.csv")

In [8]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


## Data Preprocessing


In [10]:
dataset["review"] = dataset["review"].str.lower()

In [11]:
dataset["review"][0:]

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. <br /><br />the...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    i am a catholic taught in parochial elementary...
49998    i'm going to have to disagree with the previou...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object

In [12]:

bidirdata = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

In [13]:
dataset["new reviews"] = dataset["review"].apply(lambda x : " ".join([word for word in x.split() if word not in bidirdata]))

In [14]:
def remove_tags(string):
  string = re.sub("<.*?>" , "" , string)
  return string

In [15]:
dataset["new reviews"] = dataset["new reviews"].apply(lambda x: remove_tags(x))

In [16]:
print(dataset["new reviews"][0])
print(dataset["review"][0])

one reviewers mentioned watching just 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, not show faint hearted timid. show pulls no punches regards drugs, sex violence. hardcore, classic use word.it called oz nickname given oswald maximum security state penitentary. focuses mainly emerald city, experimental section prison cells glass fronts face inwards, privacy not high agenda. em city home many..aryans, muslims, gangstas, latinos, christians, italians, irish more....so scuffles, death stares, dodgy dealings shady agreements never far away.i say main appeal show due fact goes shows wouldn't dare. forget pretty pictures painted mainstream audiences, forget charm, forget romance...oz doesn't mess around. first episode ever saw struck nasty surreal, couldn't say ready it, watched more, developed taste oz, got accustomed high levels graphic violence. not just violence, injustice (crooked guards who

In [17]:
dataset["new reviews"] = dataset["new reviews"].str.replace('[{}]'.format(string.punctuation), ' ')

In [18]:
dataset.drop('review', inplace=True, axis=1)

In [19]:
dataset.head()

,sentiment,new reviews
0,positive,one reviewers mentioned watching just 1 oz epi...
1,positive,wonderful little production the filming techn...
2,positive,thought wonderful way spend time hot summer we...
3,negative,basically family little boy jake thinks zomb...
4,positive,petter mattei s love time money visually stu...


In [20]:
def numurize(sent):
  if sent == "positive":
    return 1
  else:
    return 0

In [21]:
dataset["sentiment"] = dataset["sentiment"].apply(lambda x : numurize(x))

In [23]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sentiment    50000 non-null  int64 
 1   new reviews  50000 non-null  object
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [26]:
y = np.array(dataset["sentiment"])

In [30]:
x = list(dataset["new reviews"])

In [33]:
from sklearn.model_selection import train_test_split 

In [164]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size = 0.2)

In [165]:
print("Len of reviews in train =" , len(x_train))
print("Len of reviews in test  =" , len(x_test))

Len of reviews in train = 40000
Len of reviews in test  = 10000


In [38]:
from keras.preprocessing.text import Tokenizer

In [166]:
tokinezer = Tokenizer(num_words = 5000)
tokinezer.fit_on_texts(x_train)

words_to_index = tokinezer.word_index

In [167]:
len(words_to_index)

95660

In [46]:
!kaggle datasets download -d watts2/glove6b50dtxt

 78% 53.0M/67.7M [00:00<00:00, 71.8MB/s]
100% 67.7M/67.7M [00:00<00:00, 125MB/s] 


In [47]:
!unzip /content/glove6b50dtxt.zip

Archive:  /content/glove6b50dtxt.zip
  inflating: glove.6B.50d.txt        


In [48]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)
  return word_to_vec_map

In [50]:
word_to_vec_map = read_glove_vector('/content/glove.6B.50d.txt')

In [168]:
vocuablary_len = len(words_to_index)
embeded_len = word_to_vec_map["love"].shape[0]
max_len = 150

In [169]:
print("Number of Words =", vocuablary_len)
print("Number of Reviews =" , len(x_train))
print("Length of Embeded Vector =" , embeded_len )
print("Max Length of Words In a Review =" , max_len)

Number of Words = 95660
Number of Reviews = 40000
Length of Embeded Vector = 50
Max Length of Words In a Review = 150


## Build Model

In [108]:
from keras.models import Model
from keras.layers import Dense , Embedding , LSTM , Input , Dropout

In [170]:
#initialize embeded matrix
embed_matrix = np.zeros((vocuablary_len , embeded_len))

for word , index in words_to_index.items():
  embed_vect = word_to_vec_map.get(word)
  if embed_vect is not None:
    embed_matrix[index -1 , :] = embed_vect

In [171]:
embed_matrix.shape

(95660, 50)

In [172]:
inp = Input((max_len,))
emb = Embedding(vocuablary_len , output_dim = embed_vector_len , input_length = max_len ,
                      weights = [embed_matrix] , trainable = False)(inp)
output = LSTM(128 , return_sequences = True)(emb)
output = Dropout(0.6)(output)

output = LSTM(128 , return_sequences = True)(output)
output = Dropout(0.6)(output)

output = LSTM(128)(output)

output = Dense(1 , activation = "sigmoid")(output)

model  = Model(inputs = inp , outputs = output )

In [173]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 150)]             0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 150, 50)           4783000   
_________________________________________________________________
lstm_15 (LSTM)               (None, 150, 128)          91648     
_________________________________________________________________
dropout_10 (Dropout)         (None, 150, 128)          0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 150, 128)          131584    
_________________________________________________________________
dropout_11 (Dropout)         (None, 150, 128)          0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 128)               1315

In [117]:
from keras.preprocessing.sequence import pad_sequences

In [174]:
X_train_indices = tokinezer.texts_to_sequences(x_train)

X_train_indices = pad_sequences(X_train_indices, maxlen= max_len, padding='post')

In [175]:
X_train_indices.shape

(40000, 150)

In [176]:
model.compile(optimizer = "adam" , loss = "binary_crossentropy" , metrics = ["accuracy"])

In [177]:
model.fit(X_train_indices , y_train , batch_size = 64 , epochs = 20)

Epoch 1/20
625/625 [==============================] - 22s 29ms/step - loss: 0.6953 - accuracy: 0.5053
Epoch 2/20
625/625 [==============================] - 17s 26ms/step - loss: 0.6928 - accuracy: 0.5135
Epoch 3/20
625/625 [==============================] - 17s 27ms/step - loss: 0.6907 - accuracy: 0.5156
Epoch 4/20
625/625 [==============================] - 18s 28ms/step - loss: 0.6863 - accuracy: 0.5231
Epoch 5/20
625/625 [==============================] - 18s 28ms/step - loss: 0.6829 - accuracy: 0.5287
Epoch 6/20
625/625 [==============================] - 17s 27ms/step - loss: 0.6813 - accuracy: 0.5302
Epoch 7/20
625/625 [==============================] - 17s 27ms/step - loss: 0.6373 - accuracy: 0.6318
Epoch 8/20
625/625 [==============================] - 18s 29ms/step - loss: 0.5330 - accuracy: 0.7355
Epoch 9/20
625/625 [==============================] - 17s 28ms/step - loss: 0.4465 - accuracy: 0.7934
Epoch 10/20
625/625 [==============================] - 17s 26ms/step - loss: 0.394

In [178]:
X_test_indices = tokinezer.texts_to_sequences(x_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=max_len, padding='post')

In [180]:
model.evaluate(X_test_indices , y_test)

313/313 [==============================] - 5s 13ms/step - loss: 0.6229 - accuracy: 0.8129


[0.6228709816932678, 0.8129000067710876]

In [181]:
preds = model.predict(X_test_indices)

n = np.random.randint(0,9999)

print(x_test[n])

if preds[n] > 0.5:
  print('predicted sentiment : positive')
else: 
  print('precicted sentiment : negative')

if (y_test[n] == 1):
  print('correct sentiment : positive')
else:
  print('correct sentiment : negative')

put dvd player hit  play   will experience brief moment silence see black screen laser guided correct starting point center disc  cherish moment  make sure tylenol something  preferably pm s can fall asleep   going massive headache movie starts starring bunch big breasted girls opening actually made chuckle bit  thought good time  sure  opening sequence wee bit awkward jokes fell flat seemed like going scream ripoff  by way  sole chuckle julie strain s final comment scene   knew trouble    opening sequence terrible rock song  terrible rock song  looked dvd chapter titles saw things said  topless backyard    better sex    knew selling point movie going be and sad truth  good thing movie attractive cast  that  sadly routine slasher film throws  innovative  concept murder clubs  ends fake anyway  so  whole movie points another direction try confusing huge mystery  just adds not interesting leaves feeling like don t care characters  mean  main character movie revealed murdered innocent wom